In [1]:
from pathlib import Path
import pandas as pd
import plotly.express as px

import src.metrics_dash_osasco as mdo

pd.set_option("display.max_columns", None)
TEMPLATE = "plotly_dark"

data_path = Path().resolve().parent / "etl" / "data" / "processed"

# CAGED

In [3]:
caged = pd.read_csv(data_path / "caged.csv", sep=";", encoding="latin1")

caged_grp = caged.groupby("ano", as_index=False).agg(
    {"saldo_movimentacao": "sum"}
)
# admissoes e desligamentos
# admissoes_desligamentos = mdo.gerar_tabela_admissoes_desligamentos(caged)

# GRÁFICOS
# caged
fig_mov_month = px.bar(
    caged_grp,
    x="mes",
    y="saldo_movimentacao",
    title="Saldo Mensal de Movimentações",
    template=TEMPLATE,
)
fig_mov_month.update_xaxes(tickmode="linear", dtick="M1", tickangle=45)

C:\Users\yuri.taba\AppData\Local\Temp\ipykernel_21292\2401373776.py:1: DtypeWarning: Columns (2,13,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  caged = pd.read_csv(data_path / "caged.csv", sep=";", encoding="latin1")


ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['ano', 'saldo_movimentacao'] but received: mes

In [ ]:
# admissoes e desligamentos
fig_admissoes_desligamentos = px.bar(
    admissoes_desligamentos,
    x="mes",
    y="valor",
    color="tipo",
    barmode="group",
    title="Admissoes e Desligamentos por Mês",
    template=TEMPLATE,
)
fig_admissoes_desligamentos.update_xaxes(tickmode="linear", dtick="M1", tickangle=45)

In [ ]:
saldo_secao = (
    caged.groupby("secao_cnae_desc", as_index=False)
    .agg({"saldomovimentacao": "sum"})
    .sort_values("saldomovimentacao", ascending=True)
)

fig_saldo_secao = px.bar(
    saldo_secao,
    x="saldomovimentacao",
    y="secao_cnae_desc",
    orientation='h',
    title="Saldo de postos de trabalho em 2024 por Seção da CNAE",
    template=TEMPLATE
)
fig_saldo_secao.show()

In [ ]:
saldo_tamanho_estab = (
    caged.groupby("tamestabjan_desc", as_index=False)
    .agg({"saldomovimentacao": "sum"})
    .sort_values("saldomovimentacao", ascending=True)
)

fig_saldo_tamanho_estab = px.bar(
    saldo_tamanho_estab,
    x="saldomovimentacao",
    y="tamestabjan_desc",
    orientation='h',
    title="Saldo de postos de trabalho em 2024 por tamanho do estabelecimento",
    template=TEMPLATE
)
fig_saldo_tamanho_estab.show()

In [ ]:
saldo_idade = (
    caged.groupby("idade", as_index=False)
    .agg({"saldomovimentacao": "sum"})
    .sort_values("saldomovimentacao", ascending=True)
)

fig_saldo_idade = px.bar(
    saldo_idade,
    x="saldomovimentacao",
    y="idade",
    orientation='h',
    title="Saldo de postos de trabalho em 2024 por idade",
    template=TEMPLATE
)
fig_saldo_idade.show()

In [ ]:
salario_medio_demissoes = caged.loc[caged['saldomovimentacao'] == -1]['salario'].mean().round(2)
salario_medio_admissoes = caged.loc[caged['saldomovimentacao'] == 1]['salario'].mean().round(2)

print(f"Salário médio das *demissões* de 2024 = {salario_medio_demissoes}")
print(f"Salário médio das *admissões* de 2024 = {salario_medio_admissoes}")

In [ ]:
idade_medio_demissoes = caged.loc[caged['saldomovimentacao'] == -1]['idade'].mean().round(2)
idade_medio_admissoes = caged.loc[caged['saldomovimentacao'] == 1]['idade'].mean().round(2)

print(f"Idade médio das *demissões* de 2024 = {idade_medio_demissoes}")
print(f"Idade médio das *admissões* de 2024 = {idade_medio_admissoes}")

In [ ]:
(
    caged.loc[caged['saldomovimentacao'] == -1]
    .groupby('secao_cnae_desc', as_index=False)
    ['salario'].mean().round(2)
    .sort_values("salario", ascending=False)
)

In [ ]:
(
    caged.loc[caged['saldomovimentacao'] == 1]
    .groupby('secao_cnae_desc', as_index=False)
    ['salario'].mean().round(2)
    .sort_values("salario", ascending=False)
)

In [ ]:
# salarios errados em educação? mais de 2mi por mes, salario fixo = 13k
caged.loc[caged['secao_cnae_desc'] == 'Educação'].sort_values("salario", ascending=False).head(1)

# RAIS

In [ ]:
rais = pd.read_csv(data_path / "rais_20250211.csv", sep=";", encoding="latin1")
rais_osasco = rais.loc[rais['municipio'].str.contains("Osasco")].copy()

vinculos_secao_rais = (
    rais_osasco
    .groupby("secao_cnae", as_index=False)
    .agg({"qtd_vinculos_clt":"sum"})
    .sort_values("qtd_vinculos_clt", ascending=True)
)
fig_vinculos_secao_rais = px.bar(
    vinculos_secao_rais,
    x="qtd_vinculos_clt",
    y="secao_cnae",
    orientation='h',
    title="Saldo de postos de trabalho em 2024 por Seção da CNAE",
    template=TEMPLATE
)
fig_vinculos_secao_rais.show()

# CNAE

Estrutura: 
- 1º nível: 21 Seções
- 2º nível: 87 Divisões
- 3° nível: 285 Grupos
- 4º nível: 673 Classes
- 5º nível: 1301 Subclasses

In [ ]:
aux_path = Path().resolve().parent / "etl" / "data" / "aux_tables"
cnae = pd.read_csv(aux_path / "cnae.csv", sep=";", encoding="latin1")
cnae = cnae[
    [
        "secao_cnae",
        "divisao_cnae",
        "grupo_cnae",
        "cnae_20_classe_desc",
        "cnae_20_subclasse_desc",
    ]
].sort_values(
    [
        "secao_cnae",
        "divisao_cnae",
        "grupo_cnae",
        "cnae_20_classe_desc",
        "cnae_20_subclasse_desc",
    ]
)

cnae.loc[cnae['secao_cnae'].str.contains("Atividades administrativas e serviços")]